<a href="https://colab.research.google.com/github/JellePiepenbrock/textmining2018/blob/master/TxMM_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a streamlined version of the code used to train models for the Text Mining Project: Cross-Genre Text Author Gender Prediction Using Logistic Regression and Bidirectional LSTM

### General imports

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from google.colab import drive
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, sent_tokenize
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
from sklearn.svm import SVC
import sklearn
import spacy
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
nltk.download('punkt')
# stop_words = set(stopwords.words('dutch'))
!spacy download nl_core_news_sm
drive.mount('/content/drive')

## Read data and Create test sets

In [0]:
newsloc = "drive/My Drive/TextMining_Project/clin29/Training/GxG_News.txt"
twitterloc = "drive/My Drive/TextMining_Project/clin29/Training/GxG_Twitter.txt"
youtubeloc = "drive/My Drive/TextMining_Project/clin29/Training/GxG_YouTube.txt"

def parse_file(filelocation):

  entries = []
  identifiers = []
  genders = []

  for line in open(filelocation):
    if line.startswith("<"):
      if line.startswith("<doc id"):

        identifier = line[9:12]
        identifier = identifier.replace("\"", "")
        identifiers.append(int(identifier))

        gender = re.search(r'gender', line)
        gender = line[gender.end()+2:gender.end()+3]
        genders.append(gender)

        entry = []

      if line.startswith("</doc>"):
        entries.append(''.join(entry))

    else:
      entry.append(line)

  return entries, genders, identifiers

  
X_news, y_news, _ = parse_file(newsloc)
X_tw, y_tw, _ = parse_file(twitterloc)
X_yt, y_yt, _ = parse_file(youtubeloc)

print(len(X_news), len(y_news))
print(len(X_tw), len(y_tw))
print(len(X_yt), len(y_yt))


In [0]:
for texts, genders, outputfolder in [(X_tw, y_tw, "tw"), (X_news, y_news, "news"), (X_yt, y_yt, "yt")]:
  
  length = len(texts)
  print(length)
  y_bin = np.array([0  if k=="M" else 1 for k in genders]).reshape((len(genders)))
  print(len(y_bin))
  p = np.random.permutation(len(y_bin))

  x = pd.DataFrame(texts).iloc[p].values
#   print(x)
  y = y_bin[p]

  training_x = x[:int(np.floor(0.8*length))]
  training_y = y[:int(np.floor(0.8*length))]

  val_x = x[int(np.floor(0.8*length)):int(np.floor(0.9*length))]
  val_y = y[int(np.floor(0.8*length)):int(np.floor(0.9*length))]

  test_x = x[int(np.floor(0.9*length)):]
  test_y = y[int(np.floor(0.9*length)):]

  training = pd.DataFrame()
  val = pd.DataFrame()
  test = pd.DataFrame()

  training['label'] = training_y
  training['text']  = training_x

  val['label'] = val_y
  val['text']  = val_x

  test['label'] = test_y
  test['text']  = test_x

  # val = pd.DataFrame([training_x, training_y]).T
  # test = pd.DataFrame([training_x, training_y]).T

  # training.columns = ['label', 'text']
  # val.columns = ['label', 'text']
  # test.columns = ['label', 'text']
  print(training)
  print(val)
  print(test)
  
  assert training.shape[0] + val.shape[0] + test.shape[0] == length
#   print(len(training) + len(val) + len(test))

#   training.to_csv("drive/My Drive/TextMining_Project/pl_data/{}/train.csv".format(outputfolder), index=False, header=False)
#   val.to_csv("drive/My Drive/TextMining_Project/pl_data/{}/val.csv".format(outputfolder), index=False, header=False)
#   test.to_csv("drive/My Drive/TextMining_Project/pl_data/{}/test.csv".format(outputfolder), index=False, header=False)

# Logistic Regression

In [0]:
stopwords = list(pd.read_table('/content/drive/My Drive/TextMining_Project/stopwords-nl.txt', header=None).iloc[:,0])
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
trainedclfs = []

dataset_means = []

for dataset in ['news', 'tw', 'yt']:
  
  train = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/train.csv'.format(dataset), header=None)
  val = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/val.csv'.format(dataset), header=None)
  test = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset), header=None)

  
  
  train.columns = ['class', 'text']
  val.columns = ['class', 'text']
  test.columns = ['class', 'text']

  trainval = pd.concat([train, val])
  
  kf = KFold(n_splits=5)
  
  accuracy_folds = []
  
  for e, (train_index, val_index) in enumerate(kf.split(trainval)): 
    print("Fold number: ", e+1)
    accuracy_fold = []
    train_fold, val_fold = trainval.iloc[train_index, :], trainval.iloc[val_index, :]
  
    vectorizer = CountVectorizer(min_df=100, analyzer='char', lowercase=False, ngram_range=(1, 3))
    train_data = vectorizer.fit_transform(train_fold['text'])
    val_data = vectorizer.transform(val_fold['text'])
    test_data = vectorizer.transform(test['text'])
    
    print(train_data.shape)

    wordvectorizer = TfidfVectorizer(min_df=100, analyzer='word', lowercase=False, ngram_range=(1,3))
    train_data_word = wordvectorizer.fit_transform(train_fold['text'])
    val_data_word = wordvectorizer.transform(val_fold['text'])
    test_data_word = wordvectorizer.transform(test['text'])
  
    print(train_data_word.shape)
    train_data = np.hstack([train_data.toarray(), train_data_word.toarray()])
    val_data = np.hstack([val_data.toarray(), val_data_word.toarray()])
    test_data = np.hstack([test_data.toarray(), test_data_word.toarray()])


    scaler = StandardScaler()
    scaler.fit(train_data)

    train_scaled = scaler.transform(train_data)
    val_scaled = scaler.transform(val_data)
    test_scaled = scaler.transform(test_data)
    print(train_data.shape)

    clf = LogisticRegression(C=1)
    clf.fit(train_scaled, train_fold['class'])
    trainedclfs.append(clf)

    trainpreds = clf.predict(train_scaled)
    valpreds = clf.predict(val_scaled)
    testpreds = clf.predict(test_scaled)
    train_acc = accuracy_score(train_fold['class'], trainpreds)
    val_acc   = accuracy_score(val_fold['class'], valpreds)
    test_acc  = accuracy_score(test['class'], testpreds)

    print("Train acc: ", train_acc, "Val acc: ", val_acc, "Test acc: ", test_acc)
    
    accuracy_fold.append(train_acc)
    accuracy_fold.append(val_acc)
    accuracy_fold.append(test_acc)
    
    
    for dataset_transfer in ['news', 'tw', 'yt']:
      if dataset is not dataset_transfer:
        test_transfer = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset_transfer), header=None)
        test_transfer.columns = ['class', 'text']

        test_transfer_data = vectorizer.transform(test_transfer['text'])
        test_transfer_data_word = wordvectorizer.transform(test_transfer['text'])
        test_transfer_data = np.hstack([test_transfer_data.toarray(), test_transfer_data_word.toarray()])

        test_transfer_scaled = scaler.transform(test_transfer_data)
        transferpreds = clf.predict(test_transfer_data)
        transfer_acc = accuracy_score(test_transfer['class'], transferpreds)
        print(dataset, "->", dataset_transfer, transfer_acc)
        
        accuracy_fold.append(transfer_acc)
    accuracy_folds.append(accuracy_fold)
  dataset_means.append(accuracy_folds)

  
import pickle

with open('/content/drive/My Drive/TextMining_Project/logreg_kfold_results.pickle', 'wb') as handle:
    pickle.dump(dataset_means, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Logistic Regression with LSA inputs

In [0]:
stopwords = list(pd.read_table('/content/drive/My Drive/TextMining_Project/stopwords-nl.txt', header=None).iloc[:,0])
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
trainedclfs = []

dataset_means = []

for dataset in ['news', 'tw', 'yt']:
  
  train = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/train.csv'.format(dataset), header=None)
  val = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/val.csv'.format(dataset), header=None)
  test = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset), header=None)

  
  
  train.columns = ['class', 'text']
  val.columns = ['class', 'text']
  test.columns = ['class', 'text']

  trainval = pd.concat([train, val])
  
  kf = KFold(n_splits=5)
  
  accuracy_folds = []
  
  for e, (train_index, val_index) in enumerate(kf.split(trainval)): 
    print("Fold number: ", e+1)
    accuracy_fold = []
    train_fold, val_fold = trainval.iloc[train_index, :], trainval.iloc[val_index, :]      
    

    
    wordvectorizer = CountVectorizer(min_df=2, analyzer='word', lowercase=True, ngram_range=(1,3), stop_words=stopwords)
    train_data_word = wordvectorizer.fit_transform(train_fold['text'])
    val_data_word = wordvectorizer.transform(val_fold['text'])
    test_data_word = wordvectorizer.transform(test['text'])
    
    print(train_data_word.shape)
    # SVD represent documents and terms in vectors 
    svd_model_topics = TruncatedSVD(n_components=30, algorithm='randomized', n_iter=100, random_state=4711)

    svd_model_topics.fit(train_data_word)
    train_topics = svd_model_topics.transform(train_data_word)
    val_topics = svd_model_topics.transform(val_data_word)
    test_topics = svd_model_topics.transform(test_data_word)


    train_data = np.hstack([train_topics])
    val_data = np.hstack([val_topics])
    test_data = np.hstack([test_topics])

    scaler = StandardScaler()
    scaler.fit(train_data)

    train_scaled = scaler.transform(train_data)
    val_scaled = scaler.transform(val_data)
    test_scaled = scaler.transform(test_data)
    print(train_data.shape)

    clf = LogisticRegression(C=1)
    clf.fit(train_scaled, train_fold['class'])
    trainedclfs.append(clf)

    trainpreds = clf.predict(train_scaled)
    valpreds = clf.predict(val_scaled)
    testpreds = clf.predict(test_scaled)
    train_acc = accuracy_score(train_fold['class'], trainpreds)
    val_acc   = accuracy_score(val_fold['class'], valpreds)
    test_acc  = accuracy_score(test['class'], testpreds)

    print("Train acc: ", train_acc, "Val acc: ", val_acc, "Test acc: ", test_acc)
    
    accuracy_fold.append(train_acc)
    accuracy_fold.append(val_acc)
    accuracy_fold.append(test_acc)
    
    
    for dataset_transfer in ['news', 'tw', 'yt']:
      if dataset is not dataset_transfer:
        test_transfer = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset_transfer), header=None)
        test_transfer.columns = ['class', 'text']

#         test_transfer_data = vectorizer.transform(test_transfer['text'])
        test_transfer_data_word = wordvectorizer.transform(test_transfer['text'])
#         test_transfer_lsa = svd_model_lsa.transform(test_transfer_data)
        test_transfer_topics = svd_model_topics.transform(test_transfer_data_word)
        test_transfer_data = np.hstack([test_transfer_topics])

        test_transfer_scaled = scaler.transform(test_transfer_data)
        transferpreds = clf.predict(test_transfer_data)
        transfer_acc = accuracy_score(test_transfer['class'], transferpreds)
        print(dataset, "->", dataset_transfer, transfer_acc)
        
        accuracy_fold.append(transfer_acc)
    accuracy_folds.append(accuracy_fold)
  dataset_means.append(accuracy_folds)

import pickle

with open('/content/drive/My Drive/TextMining_Project/logreg_lsa_kfold_results.pickle', 'wb') as handle:
    pickle.dump(dataset_means, handle, protocol=pickle.HIGHEST_PROTOCOL)

# BidirectionalLSTM

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## K-folding

In [0]:
from sklearn.model_selection import KFold
dataset_means = []
for dataset in ['news', 'tw', 'yt']:
  train = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/train.csv'.format(dataset), header=None)
  val = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/val.csv'.format(dataset), header=None)
  test = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset), header=None)
  train.columns = ['class', 'text']
  val.columns = ['class', 'text']
  test.columns = ['class', 'text']

  
  trainval = pd.concat([train, val])
  
  kf = KFold(n_splits=5)
  
  accuracy_folds = []
  
  for e, (train_index, val_index) in enumerate(kf.split(trainval)): 
    print("Fold number: ", e+1)
    accuracy_fold = []
    train_fold, val_fold = trainval.iloc[train_index, :], trainval.iloc[val_index, :]
    
    def RNN():
        inputs = Input(name='inputs',shape=[max_len])
        layer = Embedding(max_words,50,input_length=max_len)(inputs)
        layer = Bidirectional(LSTM(16))(layer)
        layer = Dense(10,name='FC1')(layer)
        layer = Activation('relu')(layer)
        layer = Dropout(0.3)(layer)
        layer = Dense(1,name='out_layer')(layer)
        layer = Activation('sigmoid')(layer)
        model = Model(inputs=inputs,outputs=layer)
        return model
    if dataset=="news":
      max_len = 200
    else:
      max_len = 20

    max_words = 3000

    tok = Tokenizer(num_words=max_words)
    tok.fit_on_texts(train['text'])
    train_sequences = tok.texts_to_sequences(train_fold['text'])
    val_sequences = tok.texts_to_sequences(val_fold['text'])
    test_sequences = tok.texts_to_sequences(test['text'])

    sequences_matrix = sequence.pad_sequences(train_sequences,maxlen=max_len)
    val_sequences_matrix = sequence.pad_sequences(val_sequences,maxlen=max_len)
    test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

    model = RNN()
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

    model.fit(sequences_matrix,train_fold['class'],batch_size=128,epochs=10,
              validation_data=(val_sequences_matrix, val_fold['class']),callbacks=[EarlyStopping(patience=2, monitor='val_loss',min_delta=0.0001)])

    train_acc = model.evaluate(sequences_matrix, train_fold['class'])[1]
    val_acc = model.evaluate(val_sequences_matrix, val_fold['class'])[1]
    test_acc = model.evaluate(test_sequences_matrix, test['class'])[1]
    accuracy_fold.append(train_acc)
    accuracy_fold.append(val_acc)
    accuracy_fold.append(test_acc)
    print(train_acc, val_acc, test_acc)


    for dataset_transfer in ['news', 'tw', 'yt']:
        if dataset is not dataset_transfer:
          test_transfer = pd.read_csv('/content/drive/My Drive/TextMining_Project/pl_data/{}/test.csv'.format(dataset_transfer), header=None)
          test_transfer.columns = ['class', 'text']

          test_transfer_sequences = tok.texts_to_sequences(test_transfer['text'])
          test_transfer_sequences_matrix = sequence.pad_sequences(test_transfer_sequences,maxlen=max_len)
        
          transfer_acc = model.evaluate(test_transfer_sequences_matrix, test_transfer['class'])[1]

          print(dataset, "->", dataset_transfer, transfer_acc)
          accuracy_fold.append(transfer_acc)
          
        
    accuracy_folds.append(accuracy_fold)
  dataset_means.append(accuracy_folds)

import pickle

with open('/content/drive/My Drive/TextMining_Project/lstm_kfold_results.pickle', 'wb') as handle:
    pickle.dump(dataset_means, handle, protocol=pickle.HIGHEST_PROTOCOL)